In [49]:
import torch
import onnx
from onnx import numpy_helper

In [47]:
#!pip install onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 45.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 37.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mxnet 1.9.1 requires numpy<2.0.0,>1.16.0, but you have numpy 2.0.1 which is incompatible.
scipy 1.10.1 requires numpy<1.27.0,>=1.19.5, but you have numpy 2.0.1 which is incompatible.


In [52]:
### load ds
from face_lib.dataset_classes.lightning_datasets import MXFaceDataset

vgg_ds = MXFaceDataset("/app/datasets/faces_vgg_112x112")
# softmax_weights_path = '/app/model_weights/softmax_weight.pt'
# softmax_weights = torch.load(softmax_weights_path)
# softmax_weights_norm = torch.norm(
#             softmax_weights, dim=1, keepdim=True
#         )  # [N, 512]
# softmax_weights = (
#             softmax_weights / softmax_weights_norm
#         )

/home/erlygin/miniconda/lib/python3.9/site-packages/torchvision/transforms/v2/_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


In [56]:
vgg_ds[5000][1]

tensor(16)

In [14]:
### load model
from face_lib.models.lightning_wrappers import ResNet

backbone = ResNet(
    "iresnet50_normalized",
    "/app/model_weights/backbone/ms1mv3_arcface_r50/backbone.pth",
    learnable=False,
)

In [50]:
### load onnx model
onnx_model = onnx.load("/app/model_weights/vgg2_r50.onnx")
INTIALIZERS = onnx_model.graph.initializer
onnx_weights = {}
for initializer in INTIALIZERS:
    W = numpy_helper.to_array(initializer)
    onnx_weights[initializer.name] = W

In [51]:
onnx_weights

{'layer1.0.bn1.weight': array([-9.75188703e-32,  4.66540679e-02,  3.32212600e-32,  1.42636657e-30,
        -1.67225130e-30,  1.08468356e-30,  8.68443874e-27,  5.33340461e-27,
         4.72343101e-26,  3.98387852e-23,  2.79939137e-02,  1.89951335e-32,
         2.57309489e-02,  1.57605239e-28,  4.02086292e-33,  4.09391357e-32,
         3.68721783e-02, -1.79853353e-27,  3.15969355e-31,  2.60395790e-03,
         4.58395166e-31, -1.98711038e-27,  3.65445618e-32, -3.86139490e-19,
        -5.11454580e-33,  3.69503386e-02, -6.58578332e-34,  2.32956991e-01,
         1.21525757e-01, -2.09164085e-33,  1.42888739e-30,  1.39505282e-01,
        -2.31702738e-02,  5.72422892e-02, -2.78552139e-33,  8.58458382e-34,
        -2.44674188e-29, -3.34096670e-25,  4.24319998e-35,  3.11820080e-33,
         2.65184615e-34,  7.13315010e-32, -5.76903895e-02,  1.16919279e-01,
         1.26566634e-01,  3.17628090e-30, -5.58627095e-33,  6.12504147e-02,
         1.19067565e-01, -1.60642310e-28,  9.22528207e-02,  2.454

In [43]:
### predict emb
# x = vgg_ds[0][0][None, ...]
id = 0
x = torch.cat([vgg_ds[id][0][None, ...], vgg_ds[1][0][None, ...]])
backbone_outputs = backbone(x)
emb = backbone_outputs["feature"]

In [44]:
import numpy as np

cos = []
for weight in softmax_weights:
    cos.append(weight @ emb[0].detach().numpy())
cos = np.array(cos)

In [45]:
np.sort(cos)[::-1]

array([ 0.16963789,  0.16726862,  0.16393498, ..., -0.15562807,
       -0.15811473, -0.1590913 ], dtype=float32)

In [41]:
np.max(cos)
np.min(cos)

-0.16929848

In [37]:
# softmax_weights[vgg_ds[id][1]] @

tensor(-0.0407, grad_fn=<DotBackward0>)

In [34]:
torch.norm(softmax_weights[vgg_ds[id][1]])

tensor(1.0000)

In [21]:
torch.norm(emb, dim=-1)

tensor([1.0000, 1.0000], grad_fn=<LinalgVectorNormBackward0>)

In [16]:
x.shape

torch.Size([1, 3, 112, 112])

In [12]:
len(vgg_ds)

3137807

In [4]:
softmax_weights.shape

torch.Size([25088, 512])

In [5]:
softmax_weights

tensor([[ 8.0753e-04, -2.3171e-03,  5.5638e-04,  ...,  8.8313e-04,
         -1.0069e-03, -1.2296e-03],
        [ 1.7732e-03,  2.9120e-03, -2.9149e-03,  ..., -6.1090e-03,
          8.6583e-04,  2.1297e-04],
        [ 2.4944e-03, -9.4884e-05,  3.1424e-03,  ..., -3.0938e-03,
         -1.9145e-03,  2.5411e-03],
        ...,
        [ 3.4059e-03,  1.2368e-02, -5.6377e-03,  ...,  3.0040e-03,
         -3.6751e-03,  3.7048e-03],
        [-1.0926e-03, -6.3716e-03, -1.2018e-03,  ..., -7.8191e-04,
         -4.9295e-03, -1.8481e-03],
        [ 4.5829e-03,  6.1245e-03,  1.2969e-03,  ..., -5.7545e-03,
         -8.9320e-04, -1.6495e-03]])